<a href="https://colab.research.google.com/github/shhd-0/ML/blob/main/Random%20Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shahad164","key":"f61221b92555d23d2671c8af5615da4b"}'}

In [ ]:
import os
import shutil
from kaggle.api.kaggle_api_extended import KaggleApi

# رفع ملف kaggle.json
from google.colab import files
uploaded = files.upload()

# تأكد من أن الملف تم رفعه بنجاح
os.makedirs('/root/.config/kaggle', exist_ok=True)

# نقل الملف إلى المجلد المناسب
json_file = [f for f in uploaded.keys() if f.endswith('.json')]
if not json_file:
    raise FileNotFoundError("لم يتم رفع ملف .json")

shutil.move(json_file[0], '/root/.config/kaggle/kaggle.json')

# المصادقة وتحميل البيانات
api = KaggleApi()
api.authenticate()

# تحميل البيانات
dataset = 'charlottebennett1234/lifestyle-factors-and-their-impact-on-students'
api.dataset_download_files(dataset, path='.', unzip=True)


Saving kaggle.json to kaggle (1).json
Dataset URL: https://www.kaggle.com/datasets/charlottebennett1234/lifestyle-factors-and-their-impact-on-students


In [ ]:
import pandas as pd

# تحميل البيانات
df = pd.read_csv("student_lifestyle_dataset..csv")

# استعراض البيانات
df.head()


,Student_ID,Study_Hours_Per_Day,Extracurricular_Hours_Per_Day,Sleep_Hours_Per_Day,Social_Hours_Per_Day,Physical_Activity_Hours_Per_Day,Stress_Level,Gender,Grades
0,1,6.9,3.8,8.7,2.8,1.8,Moderate,Male,7.48
1,2,5.3,3.5,8.0,4.2,3.0,Low,Female,6.88
2,3,5.1,3.9,9.2,1.2,4.6,Low,Male,6.68
3,4,6.5,2.1,7.2,1.7,6.5,Moderate,Male,7.20
4,5,8.1,0.6,6.5,2.2,6.6,High,Male,8.78


In [ ]:
# معلومات عن الأعمدة
df.info()

# استكشاف القيم المفقودة
df.isnull().sum()

# ملء القيم المفقودة بالقيمة الأكثر تكراراً
df.fillna(df.mode().iloc[0], inplace=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Student_ID                       2000 non-null   int64  
 1   Study_Hours_Per_Day              2000 non-null   float64
 2   Extracurricular_Hours_Per_Day    2000 non-null   float64
 3   Sleep_Hours_Per_Day              2000 non-null   float64
 4   Social_Hours_Per_Day             2000 non-null   float64
 5   Physical_Activity_Hours_Per_Day  2000 non-null   float64
 6   Stress_Level                     2000 non-null   object 
 7   Gender                           2000 non-null   object 
 8   Grades                           2000 non-null   float64
dtypes: float64(6), int64(1), object(2)
memory usage: 140.8+ KB


In [ ]:
from sklearn.preprocessing import LabelEncoder

# تحويل الأعمدة الفئوية إلى أرقام
encoder = LabelEncoder()
for col in df.select_dtypes(include=['object']).columns:
    df[col] = encoder.fit_transform(df[col])


In [ ]:
# تحديد العمود الهدف (الدرجات)
target_col = "Grades"

# تحديد المتغيرات المستقلة (X) والهدف (y)
X = df.drop(columns=[target_col, "Student_ID"])
y = df[target_col]


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# تحديد المتغيرات المستقلة (X) والهدف (y)
target_col = "Grades"
X = df.drop(columns=[target_col, "Student_ID"])  # حذف العمود الهدف و 'Student_ID'
y = df[target_col]

# تقسيم البيانات إلى تدريب واختبار (70% تدريب و 30% اختبار)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# إنشاء وتدريب نموذج Random Forest Regressor
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# التنبؤ باستخدام النموذج
y_pred_rf = rf_model.predict(X_test)

# تقييم النموذج باستخدام متوسط مربع الخطأ (MSE) و R^2
mse = mean_squared_error(y_test, y_pred_rf)
r2 = r2_score(y_test, y_pred_rf)

# عرض النتائج
print("✅ Random Forest Regressor Performance:")
print(f"Mean Squared Error: {mse}")
print(f"R² (Coefficient of Determination): {r2}")



✅ Random Forest Regressor Performance:
Mean Squared Error: 0.3196023484833331
R² (Coefficient of Determination): 0.43590732597323123


In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score

# Calculate predictions
y_pred_rf = rf_model.predict(X_test)

# Calculate R² (R-squared)
r2 = r2_score(y_test, y_pred_rf)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred_rf)

# Print R² and MAE
print(f"✅ R² (Coefficient of Determination): {r2}")
print(f"✅ Mean Absolute Error (MAE): {mae}")


✅ R² (Coefficient of Determination): 0.43590732597323123
✅ Mean Absolute Error (MAE): 0.4477708333333333


In [ ]:
# Define a threshold for accuracy (e.g., if prediction is within 1 point of the actual grade)
threshold = 1
correct_predictions = abs(y_pred_rf - y_test) <= threshold
accuracy = correct_predictions.mean()

print(f"✅ Custom Regression Accuracy (within {threshold} points): {accuracy}")


✅ Custom Regression Accuracy (within 1 points): 0.9216666666666666
